# Pretraining the base embedding model 

This notebook is for pretraining the base embedding model on casia-webface. <br>
The pretrained model will then be used for fine tuning on LFW.

In [1]:
import tensorflow as tf, keras, numpy as np, matplotlib.pyplot as plt, os
from models import get_backbone_model

2024-12-18 13:49:44.636575: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734518984.652996    7685 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734518984.657985    7685 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-18 13:49:44.675249: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
casia_dir = os.path.join('Data', 'Casia Webface','casia-webface-augmented')
batch_size = 192
image_size = 128

In [ ]:
training_data, validation_data = keras.utils.image_dataset_from_directory(
    casia_dir,
    labels = 'inferred',
    label_mode = 'int',
    batch_size = batch_size,
    image_size = (image_size, image_size),
    color_mode = 'rgb',
    validation_split = 0.15,
    subset = 'both',
    shuffle = True,
    seed = 150
)




Get the base embedding model:

In [4]:
backbone_model = get_backbone_model(image_size = image_size)

In [5]:
backbone_model.summary()

Model: "backbone_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 128, 128,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 16, 16,    │  1,460,096 │ rescaling[0][0]   │
│ (Functional)        │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patches (Patches)   │ (None, 16, 8192)  │          0 │ functional[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_encoder       │ (None, 16, 128)   │  1,050,752 │ patches[0][0]     │
│ (PatchEncoder)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 16, 128)   │        256 │ patch_encoder[0]… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 16, 128)   │    527,488 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 16, 128)   │          0 │ multi_head_atten… │
│                     │                   │            │ patch_encoder[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 16, 128)   │        256 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16, 512)   │     66,048 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 16, 512)   │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16, 128)   │     65,664 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 16, 128)   │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 16, 128)   │          0 │ dropout_2[0][0],  │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 16, 128)   │        256 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 16, 128)   │    527,488 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 16, 128)   │          0 │ multi_head_atten… │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 16, 128)   │        256 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 8,837,632 (33.71 MB)

 Trainable params: 8,827,520 (33.67 MB)

 Non-trainable params: 10,112 (39.50 KB)

Add classification head:

In [6]:
classification_model = keras.models.Sequential(
    [
        backbone_model,
        keras.layers.Dropout(0.1),
        keras.layers.Dense(units = 10572, activation = 'softmax')
    ]
)

In [7]:
classification_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ backbone_model (Functional)     │ (None, 512)            │     8,837,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 10572)          │     5,423,436 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,261,068 (54.40 MB)

 Trainable params: 14,250,956 (54.36 MB)

 Non-trainable params: 10,112 (39.50 KB)

Compile the model and train:

In [8]:
optimizer = keras.optimizers.Adam(
    learning_rate = 0.00001,
)

classification_model.compile(
    optimizer = optimizer, 
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy(), keras.metrics.SparseTopKCategoricalAccuracy()]
)
weights_path = os.path.join('Data', 'Models', 'casia classification model', 'model.weights.h5')

early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 1)
learning_rate_schedule = keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.25, patience = 2)
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath = weights_path,
    verbose = 1, 
    save_weights_only = True,
    monitor = 'loss',
    save_best_only = True,
    save_freq = 1000 #save every 1000 batches

)

classification_model.load_weights(weights_path)

/home/thoalfeqar/anaconda3/envs/tf/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 464 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [11]:
classification_model.fit(
    training_data, 
    validation_data = validation_data, 
    batch_size = batch_size, 
    epochs = 150, 
    callbacks = [early_stop, learning_rate_schedule, checkpoint]
)

Epoch 1/150


I0000 00:00:1734457821.303644    5019 service.cc:148] XLA service 0x718764004f40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734457821.304145    5019 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2024-12-17 20:50:22.127069: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1734457824.728843    5019 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-12-17 20:50:28.739157: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 32 bytes spill stores, 32 bytes spill loads

2024-12-17 20:50:28.927506: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1', 280 bytes spi

  999/10861 ━━━━━━━━━━━━━━━━━━━━ 43:10 263ms/step - loss: 0.2700 - sparse_categorical_accuracy: 0.9366 - sparse_top_k_categorical_accuracy: 0.9804
Epoch 1: loss improved from inf to 0.26290, saving model to Data/Models/casia classification model/model.weights.h5
 1999/10861 ━━━━━━━━━━━━━━━━━━━━ 38:52 263ms/step - loss: 0.2633 - sparse_categorical_accuracy: 0.9385 - sparse_top_k_categorical_accuracy: 0.9809
Epoch 1: loss improved from 0.26290 to 0.25100, saving model to Data/Models/casia classification model/model.weights.h5
 2999/10861 ━━━━━━━━━━━━━━━━━━━━ 34:31 263ms/step - loss: 0.2577 - sparse_categorical_accuracy: 0.9400 - sparse_top_k_categorical_accuracy: 0.9814
Epoch 1: loss improved from 0.25100 to 0.24209, saving model to Data/Models/casia classification model/model.weights.h5
 3999/10861 ━━━━━━━━━━━━━━━━━━━━ 30:07 263ms/step - loss: 0.2527 - sparse_categorical_accuracy: 0.9412 - sparse_top_k_categorical_accuracy: 0.9818
Epoch 1: loss improved from 0.24209 to 0.23431, saving m

2024-12-17 21:38:54.500237: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_26693_0', 4 bytes spill stores, 4 bytes spill loads

2024-12-17 21:38:55.236128: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1', 280 bytes spill stores, 260 bytes spill loads

2024-12-17 21:38:55.638174: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_188', 32 bytes spill stores, 32 bytes spill loads

2024-12-17 21:38:55.663963: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_188', 80 bytes spill stores, 84 bytes spill loads

2024-12-17 21:38:55.779188: I external/local_xla/xla/stream_ex

10861/10861 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - loss: 0.2338 - sparse_categorical_accuracy: 0.9453 - sparse_top_k_categorical_accuracy: 0.9834

2024-12-17 21:39:20.351979: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2543', 240 bytes spill stores, 240 bytes spill loads

2024-12-17 21:39:20.563232: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2543', 744 bytes spill stores, 440 bytes spill loads

2024-12-17 21:41:56.348678: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_58', 80 bytes spill stores, 80 bytes spill loads

2024-12-17 21:41:56.579723: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1', 280 bytes spill stores, 260 bytes spill loads

2024-12-17 21:41:56.720120: I external/local_xla/xla/stream

10861/10861 ━━━━━━━━━━━━━━━━━━━━ 3121s 281ms/step - loss: 0.2338 - sparse_categorical_accuracy: 0.9453 - sparse_top_k_categorical_accuracy: 0.9834 - val_loss: 0.2446 - val_sparse_categorical_accuracy: 0.9530 - val_sparse_top_k_categorical_accuracy: 0.9795 - learning_rate: 5.0000e-05
Epoch 2/150
  138/10861 ━━━━━━━━━━━━━━━━━━━━ 48:47 273ms/step - loss: 0.1500 - sparse_categorical_accuracy: 0.9600 - sparse_top_k_categorical_accuracy: 0.9910
Epoch 2: loss improved from 0.21530 to 0.15626, saving model to Data/Models/casia classification model/model.weights.h5
 1138/10861 ━━━━━━━━━━━━━━━━━━━━ 43:26 268ms/step - loss: 0.1578 - sparse_categorical_accuracy: 0.9600 - sparse_top_k_categorical_accuracy: 0.9905
Epoch 2: loss did not improve from 0.15626
 2138/10861 ━━━━━━━━━━━━━━━━━━━━ 38:49 267ms/step - loss: 0.1588 - sparse_categorical_accuracy: 0.9599 - sparse_top_k_categorical_accuracy: 0.9905
Epoch 2: loss did not improve from 0.15626
 3138/10861 ━━━━━━━━━━━━━━━━━━━━ 34:24 267ms/step - loss:

In [9]:
# remove the classification head and save the weights of the backbone

backbone = keras.models.Model(
    inputs = classification_model.layers[0].input,
    outputs = classification_model.layers[0].output
)

backbone.save_weights(
    os.path.join('Data', 'Models', 'backbone after classification', 'model.weights.h5')
)